LOADING THE DATA

In [14]:
import pandas as pd 
import numpy as np
from pathlib import Path

# Load processed data
path = Path("../data/processed/books.csv")
df = pd.read_csv(path)
df.head()

,work_key,title,first_publish_year,authors,source_subject,subjects
0,/works/OL138052W,Alice's Adventures in Wonderland,1865,Lewis Carroll,fantasy,"Alice (fictitious character : carroll), fictio..."
1,/works/OL18417W,The Wonderful Wizard of Oz,1899,L. Frank Baum,fantasy,Witches; Toy and movable books; Spanish langua...
2,/works/OL24034W,Treasure Island,1880,Robert Louis Stevenson,fantasy,Fiction; Treasure Island (Imaginary place); Tr...
3,/works/OL20600W,Gulliver's Travels,1726,Jonathan Swift,fantasy,YA; Young adult; Juvenile; Fiction; Fantasy; U...
4,/works/OL259010W,A Midsummer Night's Dream,1600,William Shakespeare,fantasy,Drama; Courtship; Plays; Hippolyta (Greek myth...


PREPARING FOR RECOMMEND FOR ONE BOOK

In [15]:
# Create a combined text field for vectorization
df["text"] = (
    df["title"].fillna("") + " " +
    df["authors"].fillna("") + " " +
    df["source_subject"].fillna("") + " " +
    df["subjects"].fillna("")
)

df[["title", "text"]].head()
from sklearn.feature_extraction.text import TfidfVectorizer
# Vectorize the text data
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=10000
)

X = vectorizer.fit_transform(df["text"])
X.shape
from sklearn.metrics.pairwise import cosine_similarity
# Compute cosine similarity matrix
similarity = cosine_similarity(X)
similarity.shape


(9252, 9252)

RECOMMEND FUNCTION

In [16]:
# Recommendation function for a single book
def recommend(title, df, similarity, top_k=5):
    if title not in df["title"].values:
        return "Book not found"

    idx = df.index[df["title"] == title][0]
    scores = list(enumerate(similarity[idx]))
    
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    recommendations = []
    for i, score in scores[1: top_k + 1]:
        recommendations.append(df.iloc[i]["title"])

    return recommendations



EXAMPLE

In [17]:
# Example usage
book = df["title"].sample(1).values[0]
print(f"Recommendations for '{book}':")
rec= recommend(book, df, similarity)
print(rec)

Recommendations for 'Armageddon':
['The Dangerous Days of Daniel X', 'Demons and druids', 'Watch the skies', 'Game over', 'Planet of the Damned']


IMPROVING WEIGHT AND PREPARING AGAIN

In [ ]:
# Create a weighted text field with more emphasis on source_subject
df["text_weighted"] = (
    df["title"].fillna("") + " " +
    df["authors"].fillna("") + " " +
    df["source_subject"].fillna("") + " " +
    df["subjects"].fillna("") 
)

df[["title", "text_weighted"]].head()
# Vectorize the weighted text data for improved recommendations
X_w = vectorizer.fit_transform(df["text_weighted"])
X_w.shape
# Compute cosine similarity matrix for weighted text
similarity_w = cosine_similarity(X_w)



EXAMPLE FOR IMPROVED

In [19]:
# Example usage with weighted similarity
book = df["title"].sample(1).values[0]
print(f"Recommendations for '{book}':")
print(recommend(book, df, similarity_w))


Recommendations for 'The Mystery in the Fortune Cookie':
['The Mystery Of The Haunted Boxcar', 'The Finders Keepers Mystery', 'The Ice Cream Mystery', 'The Comic Book Mystery', 'The Tattletale Mystery']


GETTING RATING DATA FROM rating.csv FILE

In [32]:
import pandas as pd
from pathlib import Path

ratings_path = Path("../data/ratings.csv")
print(f"Reading ratings from: {ratings_path}")
if not ratings_path.exists():
    raise FileNotFoundError(f"File not found: {ratings_path}")
ratings = pd.read_csv(ratings_path)
ratings.head()

Reading ratings from: ../data/ratings.csv


,user_id,work_key,rating
0,1,/works/OL20600W,5
1,1,/works/OL82563W,5
2,1,/works/OL1449046W,5
3,1,/works/OL893502W,5


TEST FOR MULTIPLE BOOK AND WITH RATINGS

In [29]:
# work_key to index mapping
work_to_idx = {
    k: i
    for i, k in enumerate(df["work_key"])
}
list(work_to_idx.items())[:5]

# User-test ratings
user_id = 1
user_ratings = ratings[ratings["user_id"] == user_id]
user_ratings

import numpy as np
item_vectors = []
weights = []
# Building user profile vector
for row in user_ratings.itertuples(index=False):
    work_key = row.work_key
    rating = row.rating
    idx = work_to_idx.get(work_key)

    v = X_w[idx].toarray()[0]
    item_vectors.append(v)
    weights.append(rating)
    
item_matrix = np.vstack(item_vectors)
print(item_matrix.shape)
weights = np.array(weights)
print(weights.shape)
user_profile = np.average(item_matrix, axis=0, weights=weights)
user_profile.shape
from sklearn.metrics.pairwise import cosine_similarity
# Compute similarity scores between user profile and all items
scores = cosine_similarity(user_profile.reshape(1, -1), X_w).ravel()
scores.shape  # (734,)


# Kullanıcının zaten rated ettiği kitapların indexleri
rated_idx = {work_to_idx[wk] for wk in user_ratings["work_key"]}

# (index, skor) listesi
idx_scores = list(enumerate(scores))

# zaten rated olanları filtrele
idx_scores = [
    (i, s) for i, s in idx_scores
    if i not in rated_idx
]

# yüksekten düşüğe sırala
idx_scores = sorted(idx_scores, key=lambda x: x[1], reverse=True)

top_k = 10
top_idx = [i for i, s in idx_scores[:top_k]]

df.iloc[top_idx][["title", "authors", "source_subject"]]

    

(4, 10000)
(4,)


,title,authors,source_subject
75,The Voyages of Doctor Dolittle,Hugh Lofting,fantasy
45,Harry Potter and the Chamber of Secrets,J. K. Rowling,fantasy
55,Harry Potter and the Order of the Phoenix,J. K. Rowling,fantasy
50,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,fantasy
56,Harry Potter and the Goblet of Fire,J. K. Rowling,fantasy
649,Doctor Dolittle's caravan,Hugh Lofting,fantasy
142,Doctor Dolittle in the moon,Hugh Lofting,fantasy
711,Gulliver's travels in Lilliput and Brobdingnag,Jonathan Swift,fantasy
145,Doctor Dolittle's post office,Hugh Lofting,fantasy
179,Doctor Dolittle's garden,Hugh Lofting,fantasy


FUNCTION FOR MULTIPLE BOOKS AND ITS RATINGS FOR SPECIFIC USER

In [30]:
# Recommendation function for a user based on their ratings
def recommend_for_user(user_id, df, X_w, ratings, work_to_idx, top_k):
    user_ratings = ratings[ratings["user_id"] == user_id]
    if user_ratings.empty:
        return df.sample(top_k)[["title", "authors", "source_subject"]]
    item_vectors = []
    weights = []
    for row in user_ratings.itertuples(index=False):
        work_key = row.work_key
        rating = row.rating
        idx = work_to_idx.get(work_key)

        v = X_w[idx].toarray()[0]
        item_vectors.append(v)
        weights.append(rating)
    item_matrix = np.vstack(item_vectors)
    weights = np.array(weights)
    user_profile = np.average(item_matrix, axis=0, weights=weights)

    similarity_scores = cosine_similarity(user_profile.reshape(1, -1), X_w).ravel()
    rated_idx = {work_to_idx[wk] for wk in user_ratings["work_key"]}
    print(df[df["work_key"].isin(user_ratings["work_key"])].title)
    idx_scores = list(enumerate(similarity_scores))
    idx_scores = [
        (i, s) for i, s in idx_scores
        if i not in rated_idx 
    ]
    top_idx = sorted(idx_scores, key=lambda x: x[1], reverse=True)
    top_idx = [i for i, s in top_idx[:top_k]]
    return df.iloc[top_idx][["title", "authors", "source_subject"]]


In [33]:
recommend_for_user(1, df, X_w, ratings, work_to_idx, top_k=20)

3                             Gulliver's Travels
39      Harry Potter and the Philosopher's Stone
61                  The Story of Doctor Dolittle
1230                            Heretics of Dune
Name: title, dtype: object


,title,authors,source_subject
75,The Voyages of Doctor Dolittle,Hugh Lofting,fantasy
45,Harry Potter and the Chamber of Secrets,J. K. Rowling,fantasy
55,Harry Potter and the Order of the Phoenix,J. K. Rowling,fantasy
50,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,fantasy
56,Harry Potter and the Goblet of Fire,J. K. Rowling,fantasy
649,Doctor Dolittle's caravan,Hugh Lofting,fantasy
142,Doctor Dolittle in the moon,Hugh Lofting,fantasy
711,Gulliver's travels in Lilliput and Brobdingnag,Jonathan Swift,fantasy
1242,Chapterhouse Dune,Frank Herbert,science_fiction
145,Doctor Dolittle's post office,Hugh Lofting,fantasy
